In [3]:
from gtfslib import GTFS
gtfs = GTFS("data/dart_gtfs_silverlineopening.zip")

import pandas as pd
from pathlib import Path

export_folder = Path("export/silverline_schedules")
export_folder.mkdir(parents=True, exist_ok=True)

stations = {
    "DFW Terminal B": "33473",
    "DFW Airport North": "33474",
    "Cypress Waters": "33595",
    "Downtown Carrollton": "34292",
    "Addison": "33596",
    "Knoll Trail": "33597",
    "UTD": "33598",
    "CityLine Bush": "26895",
    "12th Street": "34293",
    "Shiloh Road": "33600",
}
dates = [("openingday", "20251025"), ("weekday", "20251028"), ("weekend", "20251108")]

Generate departure timetable CSVs for each station

In [ ]:
for folder, date in dates:
    (export_folder / folder).mkdir(exist_ok=True)
    timetables = pd.DataFrame()
    for name, stop_id in stations.items():
        timetable = gtfs.build_stop_timetable(stop_id, [date])
        timetable["station"] = name
        timetable = timetable[["station", "trip_headsign", "departure_time", "block_id"]]
        timetable.columns = ["Station", "Destination", "Departure Time", "Train ID"]
        # CityLine Bush also serves light rail, so export all trains for that station separately
        if name == "CityLine Bush":
            timetable_all = timetable.copy()
            timetable_all.to_csv(export_folder / folder / f"{name.lower().replace(' ', '_')}_{folder}_{date}_alltrains.csv", index=False)
            timetable = timetable[timetable["Destination"].str.contains("SILVER LINE")]
        timetable.to_csv(export_folder / folder / f"{name.lower().replace(' ', '_')}_{folder}_{date}.csv", index=False)
        timetables = pd.concat([timetables, timetable])
    timetables.to_csv(export_folder / folder / f"all_stations_combined_{folder}_{date}.csv", index=False)

Generate combined destination timetables

In [7]:
from collections import defaultdict
import csv

def parse_time(t):
    h, m, _ = map(int, t.split(":"))
    if h >= 24:
        h -= 24
    suffix = "AM" if h < 12 else "PM"
    h %= 12
    if h == 0:
        h = 12
    return f"{h}:{m:02d} {suffix}"

for folder, date in dates:
    departures = defaultdict(lambda: defaultdict(list))
    for row in csv.DictReader((export_folder / folder / f"all_stations_combined_{folder}_{date}.csv").open()):
        departures[row["Destination"]][row["Station"]].append(row["Departure Time"])

    for destination, stations in departures.items():
        if "WEST" in destination:
            destination = "westbound"
        else:
            destination = "eastbound"
        rows = []
        for station, times in stations.items():
            row = [station] + list(map(parse_time, times))
            rows.append(row)
        if destination == "westbound":
            rows = rows[::-1]
        pd.DataFrame(rows).transpose().to_csv(export_folder / f"destination_{destination}_{folder}_{date}.csv", index=False, header=False)